In [24]:
import math
import random
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from utils import toTorchTensers, NormalizeData

base_url = r'.'

data_url = rf'{base_url}/data/Data.xlsx'
data_egypt = pd.read_excel(data_url, sheet_name='Egypt')
data_vietnam = pd.read_excel(data_url, sheet_name='Vietnam')

device = 'cpu'
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)


In [25]:
class AutoEncoder(nn.Module):
  def __init__(self):
    super().__init__()

    self.opti = None
    self.crit = nn.MSELoss()
    self.encoder = nn.Sequential(
        nn.Linear(12, 8),
        nn.ReLU(),
        nn.Linear(8, 6),
        nn.ReLU(),
        nn.Linear(6, 3),
        # nn.Sigmoid(),
    )
    self.decoder = nn.Sequential(
        nn.Linear(3, 6),
        nn.ReLU(),
        nn.Linear(6, 8),
        nn.ReLU(),
        nn.Linear(8, 12),
        nn.Sigmoid(),
    )

    self.init()

  def init(self):
    self.opti = optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-5)

  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

  def fit(self, data_x, data_y, epoches=100):

    for epoche in range(epoches):
      epoche_start_time = timer()

      for i in range(len(data_x)):
        recon = self(data_x[i])
        loss = self.crit(recon, data_y[i])

        self.opti.zero_grad()
        loss.backward()
        self.opti.step()

      epoche_exec_time = f"{(timer() - epoche_start_time):.1f}"
      print(
          f'epoche: {epoche}, loss: {loss.item():.8f}, execution time: {epoche_exec_time}s'
      )

In [32]:
# data
DATA = data_egypt
DATA = NormalizeData(DATA)
DATA = DATA[DATA.Flood == 1]
DATA = DATA.to_numpy()

DATA_X = DATA[:, 1:]

[DATA_X] = toTorchTensers(DATA_X, device=device)

In [33]:
loaded_model = AutoEncoder().to(device)
loaded_model.load_state_dict(
    torch.load(f'{base_url}/models/X2_auto_model_06-04-2022_23-27-39.pt')
)
loaded_model.eval()
print('model loaded')


model loaded


In [36]:
new_classes = []
for input in DATA_X:
  encoded = loaded_model.encoder(input)
  encoded = (nn.Sigmoid())(encoded)
  encoded = encoded.detach().cpu().numpy()
  encoded = np.argmax(encoded) 
  new_classes.append(encoded)

print(f'{new_classes}', end=' ')


[2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 

In [72]:
from datetime import datetime
DATA = data_egypt
DATA['NewFlood'] = np.zeros(DATA.shape[0])
DATA = DATA.to_numpy()

i = 0
for ii in range(DATA.shape[0]):
  if DATA[ii][0] == 1:
    DATA[ii][-1] = new_classes[i]
    i+=1

DATA_TO_SAVE = data_egypt
DATA_TO_SAVE['NewFlood'] = DATA[:, -1]
#
current_datetime = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
path = rf'{base_url}/data/New_Class_Data_{current_datetime}.xlsx'
DATA_TO_SAVE.to_excel(path)

print(f'✅ data saved to "New_Class_Data_{current_datetime}.xlsx"')